In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from tqdm import tqdm
from model import *
from coin_estimator_utils import data_transforms

In [ ]:
data_transforms = transforms.Compose([
    transforms.RandomRotation(360),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing to [0, 1]
])

train_dataset = torchvision.datasets.ImageFolder(root="coin_dataset/training/", transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)

In [ ]:
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("mps")
model = CoinClassifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
num_epochs = 25

for epoch in range(num_epochs):
    epoch_loss = 0.0  # Accumulate loss here
    
    # Create a tqdm object for the training loop
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=True)
    
    for i, (inputs, labels) in loop:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()  # Add current batch's loss
        
        # Update tqdm progress bar
        loop.set_description(f"Epoch [{epoch + 1}/{num_epochs}]")
        loop.set_postfix(loss=loss.item())

    average_epoch_loss = epoch_loss / len(train_loader)
    print(f"Epoch [{epoch + 1}/{num_epochs}] Average Loss: {average_epoch_loss:.4f}")
    
    # Save the model checkpoint after each epoch
    torch.save(model.state_dict(), "checkpoints/coin_estimator.pth")

print("Training completed!")
